In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
lgas = gpd.read_file("../Data/Training Data/LGA_2024_AUST_GDA2020.zip")
council_names = pd.read_csv("../Data/Training Data/ALGA Mail List.csv", encoding="latin1")

grants = pd.read_csv("../Data/Training Data/Grants Councils.csv")

In [3]:
geometry = lgas[["LGA_CODE24", "geometry"]]

lgas = pd.DataFrame(lgas.drop(columns=["geometry"]))

In [4]:
lgas = lgas[~lgas["LGA_NAME24"].str.contains(r"Unincorporated|Offshore|No usual address|Outside Australia|Unincorp. Other Territories")]

In [5]:
council_names.dropna(subset="COUNCIL", inplace=True)

In [6]:
lgas["Council Name"] = lgas["LGA_NAME24"].apply(lambda x: [i for i in council_names["COUNCIL"] if x in i])
lgas["Council Name"] = lgas["Council Name"].apply(lambda x: x[0] if len(x) == 1 else x)

In [7]:
replace_dict = {
    "Anangu Pitjantjatjara Yankunytjatjara" : pd.NA, # HAS TO BE REPLACED
    "Augusta Margaret River" : "Shire of Augusta-Margaret River",
    "Bayside (NSW)" : "Bayside Council",
    "Bayside (Vic.)" : "Bayside City Council",
    "Blackall Tambo" : "Blackall-Tambo Regional Council",
    "Campbelltown (NSW)" : "Campbelltown City Council",
    "Campbelltown (SA)" : "Campbelltown City Council (SA)",
    "Central Coast (NSW)" : "Central Coast Council",
    "Central Coast (Tas.)" : "Central Coast Council (Tas.)",
    "Central Highlands (Qld)" : "Central Highlands Regional Council",
    "Central Highlands (Tas.)" : "Central Highlands Council",
    "Cocos Islands" : "Shire of Cocos (Keeling) Islands",
    "Darwin Waterfront Precinct" : "City of Darwin",
    "Derby-West Kimberley" : "Shire of Derby/West Kimberley",
    "Flinders (Qld)" : "Flinders Shire Council",
    "Flinders (Tas.)": "Flinders Council",
    "Glamorgan-Spring Bay": "Glamorgan Spring Bay Council",
    "Hunters Hill" : "Hunter's Hill Council",
    "Kingston (SA)" : "Kingston District Council",
    "Kingston (Vic.)" : "City of Kingston",
    "Latrobe (Tas.)" : "Latrobe Council",
    "Latrobe (Vic.)" : "Latrobe City Council",
    "Maralinga Tjarutja" : pd.NA, # HAS TO BE REPLACED
    "Mid-Coast" : "Mid Coast Council",
    "Mount Marshall" : "Shire of Mt Marshall",
    "Nambucca Valley" : "Nambucca Shire Council",
    "Roxby Downs" : "Roxby Council",
    "Serpentine-Jarrahdale" : "Shire of Serpentine Jarrahdale",
    "Waratah-Wynyard" : "Waratah - Wynyard Council",
    "Weipa" : "Weipa Town Authority"
}

lgas.loc[lgas["LGA_NAME24"].isin(replace_dict.keys()), "Council Name"] = lgas.loc[lgas["LGA_NAME24"].isin(replace_dict.keys()), "LGA_NAME24"].apply(lambda x: replace_dict[x])

In [8]:
lgas = lgas[lgas["Council Name"].notna()]

In [9]:
lgas["Council Name"] = lgas["Council Name"].apply(lambda x: [i for i in x if i not in lgas["Council Name"].values][0] if type(x) == list else x)

In [10]:
lgas["LGA_CODE24"] = lgas["LGA_CODE24"].astype(str)

In [11]:
lgas = lgas.merge(geometry, on="LGA_CODE24")

In [12]:
grants.loc[(grants["Recipient State/Territory"] == "TAS") & (grants["Assigned LGA"] == "Central Coast Council"), "Assigned LGA"] = "Central Coast Council (Tas.)"

In [13]:
grants_geo = lgas.merge(grants, left_on="Council Name", right_on="Assigned LGA")

In [15]:
grants_geo.columns

Index(['LGA_CODE24', 'LGA_NAME24', 'STE_CODE21', 'STE_NAME21', 'AUS_CODE21',
       'AUS_NAME21', 'AREASQKM', 'LOCI_URI21', 'Council Name', 'geometry',
       'Agency', 'Parent GA ID', 'GA ID', 'Status', 'Publish Date',
       'Variation Date', 'Approval Date', 'Start Date', 'End Date',
       'Value (AUD)', 'Variation Value (AUD)', 'Variation Reason',
       'PBS Program Name', 'Grant Program', 'Grant Activity', 'Purpose',
       'One-off/Ad hoc', 'Aggregate', 'Aggregate Reason', 'Aggregate Number',
       'Selection Process', 'Category', 'Internal Reference ID', 'GO ID',
       'Confidentiality - Contract', 'Confidentiality Reason(s) - Contract',
       'Confidentiality - Outputs', 'Confidentiality Reason(s) - Outputs',
       'Recipient Name', 'Recipient ABN', 'Recipient Suburb',
       'Recipient Town/City', 'Recipient Postcode',
       'Recipient State/Territory', 'Recipient Country',
       'Delivery State/Territory', 'Delivery Postcode', 'Delivery Country',
       'Contact Name'

In [16]:
grants_geo.geometry = grants_geo.geometry.set_precision(grid_size=0.01)

In [ ]:
grants_geo.explore()

In [18]:
columns = [
    'LGA_CODE24', 'LGA_NAME24', 'Council Name', 'Agency',
       'Parent GA ID', 'GA ID', 'Status',
       'Approval Date', 'Start Date', 'End Date', 'Value (AUD)',
       'Variation Value (AUD)', 'Variation Reason', 'PBS Program Name',
       'Grant Program', 'Grant Activity', 'Purpose', 'One-off/Ad hoc',
       'Selection Process', 'Category', 'GO ID',
       'Recipient Name', 'Assigned LGA', 'geometry'
]

In [19]:
grants_geo = grants_geo[columns]

In [20]:
grants_geo.to_file("../Data/Training Data/Grants Councils.geojson", driver="GeoJSON")

In [21]:
grants_geo.to_file("../Data/Training Data/Grants Councils.shp")   

/tmp/ipykernel_4100/2775157018.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grants_geo.to_file("../Data/Training Data/Grants Councils.shp")
/home/spulick/miniforge3/envs/grants/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Council Name' to 'Council Na'
  ogr_write(
/home/spulick/miniforge3/envs/grants/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Parent GA ID' to 'Parent GA'
  ogr_write(
/home/spulick/miniforge3/envs/grants/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Approval Date' to 'Approval D'
  ogr_write(
/home/spulick/miniforge3/envs/grants/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Value (AUD)' to 'Value (AUD'
  ogr_write(
/home/spulick/miniforge3/envs/grants/lib/python3.10/site-packages/pyogrio/raw.py:7

# Unincorporated

In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
lgas = gpd.read_file("../Data/Training Data/LGA_2024_AUST_GDA2020.zip")

In [4]:
lgas = lgas[lgas["LGA_NAME24"].str.contains("Unincorp")]

In [5]:
lgas

AttributeError: 'NoneType' object has no attribute 'copy'

,LGA_CODE24,LGA_NAME24,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM,LOCI_URI21,geometry
128,19399,Unincorporated NSW,1,New South Wales,AUS,Australia,93208.9194,https://linked.data.gov.au/dataset/asgsed3/LGA...,"MULTIPOLYGON (((151.25862 -33.85965, 151.25865..."
210,29399,Unincorporated Vic,2,Victoria,AUS,Australia,283.1903,https://linked.data.gov.au/dataset/asgsed3/LGA...,"MULTIPOLYGON (((146.29286 -39.15778, 146.29341..."
363,49399,Unincorporated SA,4,South Australia,AUS,Australia,619426.0858,https://linked.data.gov.au/dataset/asgsed3/LGA...,"MULTIPOLYGON (((136.11505 -35.33794, 136.11491..."
556,79399,Unincorporated NT,7,Northern Territory,AUS,Australia,19777.3595,https://linked.data.gov.au/dataset/asgsed3/LGA...,"MULTIPOLYGON (((130.74789 -12.42749, 130.74985..."
559,89399,Unincorporated ACT,8,Australian Capital Territory,AUS,Australia,2358.1330,https://linked.data.gov.au/dataset/asgsed3/LGA...,"POLYGON ((149.06239 -35.1591, 149.09134 -35.14..."
562,99399,Unincorp. Other Territories,9,Other Territories,AUS,Australia,105.8806,https://linked.data.gov.au/dataset/asgsed3/LGA...,"MULTIPOLYGON (((167.94747 -29.12757, 167.94748..."


In [6]:
lgas = lgas[["LGA_CODE24", "LGA_NAME24", "geometry"]]

In [8]:
lgas.geometry = lgas.geometry.set_precision(grid_size=0.01)

/home/spulick/miniforge3/envs/grants/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [10]:
lgas.to_file("../Data/Training Data/Unincorp.geojson", driver="GeoJSON")